## 过度拟合例子

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.pipeline as pl
import sklearn.preprocessing as sp
import sklearn.linear_model as lm

# 生成模拟数据
x = np.arange(0,6,0.2).reshape(-1,1) 
y = -5*x**2 + x*10 + 1 + (np.random.randn(len(x))*10).reshape(-1,1) # 这里我们使用一个二次函数加上一个噪声模拟训练目标
x = np.delete(x, [2,3,5,8], axis = 0) # 删除其中三个点
y = np.delete(y, [2,3,5,8], axis = 0)

plt.scatter(x, y)
plt.show()

# 可以互动的版本，最高次数可调
def get_fited_model(n):
    # 岭回归器
    lmr = lm.Ridge(alpha=1)

    #线性回归器
    lml = lm.LinearRegression()

    model_lmr = pl.make_pipeline(
                    sp.PolynomialFeatures(n),  # 多项式特征扩展器
                    lmr)      # 岭回归器
    model_lmr.fit(x, y)

    model_lml = pl.make_pipeline(
                    sp.PolynomialFeatures(n),  # 多项式特征扩展器
                    lml)      # 线性回归器
    model_lml.fit(x, y)
    return model_lmr, model_lml

def plot_result(n):
    n = int(n)
    model_lmr, model_lml = get_fited_model(n)
    px = np.arange(0,5.9,0.1).reshape(-1,1)
    pred_py_lmr = model_lmr.predict(px)
    pred_py_lml = model_lml.predict(px)
    # 图1，未正则化结果
    plt.subplot(1,2,1)
    plt.plot(px, pred_py_lml, c='orangered', label='PolyFit Line')
    plt.scatter(x, y)
    plt.title("未使用正则化")

    # 图2，正则化结果
    plt.subplot(1,2,2)
    plt.plot(px, pred_py_lmr, c='orangered', label='PolyFit Line')
    plt.scatter(x, y)
    plt.title("使用正则化")
    
    
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# 拖拽修改参数 a, b, c 的值来观察绘图的变化情况
interact(plot_result,
         n=widgets.FloatSlider(min=1, max=20, step=1.0, value=1),
        );

ModuleNotFoundError: No module named 'sklearn'

## 二分类任务例子

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# 每个类别样本个数
point_counts = 50
X_1 = np.random.normal(2,2,size=(point_counts,2)) # 使用均值为2，方差为1.5的正态分布生成1类点
y_1 = np.zeros((point_counts,1))

X_2 = np.random.normal(-2,2,size=(point_counts,2)) # 使用均值为-2，方差为1.5的正态分布生成2类点
y_2 = np.ones((point_counts,1))

# 使用颜色区分两种点
plt.scatter(X_1[:,0],X_1[:,1],c='green')
plt.scatter(X_2[:,0],X_2[:,1],c='red')

# 可交互版本，调节多项式的特征
def fit_logits_model(n):
    X_train = np.vstack((X_1,X_2))
    y_train = np.vstack((y_1,y_2))

    #定义多项式回归,degree的值可以调节多项式的特征
    poly_reg = PolynomialFeatures(degree=n)
    #特征处理
    X_train = poly_reg.fit_transform(X_train)

    # 无正则化的多项式逻辑回归
    log_reg = LogisticRegression(penalty='none',tol=1e-6,solver='lbfgs')
    # 带l2正则化的逻辑回归
    log_reg_withl2 = LogisticRegression(penalty='l2',tol=1e-6,C=1e-5,solver='lbfgs')

    # 进行训练
    log_reg.fit(X_train, y_train)
    log_reg_withl2.fit(X_train,y_train)

    return log_reg, log_reg_withl2, poly_reg

def plot_logits_result(n):
    n = int(n)
    log_reg, log_reg_withl2, poly_reg = fit_logits_model(n)
    # 生成网格矩阵
    xx, yy = np.meshgrid(np.arange(-8, 8, 0.02),
                        np.arange(-8, 8, 0.02))
    z = log_reg.predict(poly_reg.fit_transform(np.c_[xx.ravel(), yy.ravel()]))
    z = z.reshape(xx.shape)
    z_wihtl2 = log_reg_withl2.predict(poly_reg.fit_transform(np.c_[xx.ravel(), yy.ravel()]))
    z_wihtl2 = z_wihtl2.reshape(xx.shape)

    plt.subplot(1,2,1)
    cs = plt.contourf(xx, yy, z)
    plt.scatter(X_1[:,0],X_1[:,1])
    plt.scatter(X_2[:,0],X_2[:,1],c='red')
    plt.title("未使用正则化")

    plt.subplot(1,2,2)
    cs = plt.contourf(xx, yy, z_wihtl2)
    plt.scatter(X_1[:,0],X_1[:,1])
    plt.scatter(X_2[:,0],X_2[:,1],c='red')
    plt.title("使用正则化")

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# 拖拽修改参数 a, b, c 的值来观察绘图的变化情况
interact(plot_logits_result,
         n=widgets.FloatSlider(min=3, max=10, step=1.0, value=1),
        );    

ModuleNotFoundError: No module named 'sklearn'